In [36]:
%reload_ext autoreload
%autoreload 2

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
from glob import glob
import wave
import math

import numpy as np

import librosa

import tensorflow as tf
from model.fp.melspec.melspectrogram import get_melspec_layer
from run import load_config
from model.utils.audio_utils import load_audio

In [37]:
DISCOTUBE = "/mnt/projects/discotube/discotube-2023-03/audio-new/audio/"
audio_directories = os.listdir(DISCOTUBE)
audio_directory = audio_directories[0]
mp4_paths = glob(os.path.join(DISCOTUBE, audio_directory, "*.mp4"))
mp4_path = mp4_paths[0]

In [38]:
audio_dir = "/home/oaraz/nextcore/afp/neural-audio-fp-dataset/music/test-dummy-db-100k-full/from_fma_large10k-30s_for_mini/"
audio_paths = glob(os.path.join(audio_dir, "*/*.wav"))
print(len(audio_paths))

9978


In [4]:
cfg = load_config("default")
melspec_layer = get_melspec_layer(cfg)

print(cfg["MODEL"])

fs = cfg["MODEL"]["FS"]

cli: Configuration from ./config/default.yaml


2023-06-14 14:56:38.149779: E tensorflow/stream_executor/cuda/cuda_driver.cc:313] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


{'FEAT': 'melspec', 'FS': 8000, 'DUR': 1.0, 'HOP': 0.5, 'STFT_WIN': 1024, 'STFT_HOP': 256, 'F_MIN': 300.0, 'F_MAX': 4000.0, 'N_MELS': 256, 'EMB_SZ': 128, 'BN': 'layer_norm2d'}


In [15]:
for audio_path in audio_paths:
    audio = load_audio(audio_path, 0.0, 0.0, 10.0, 0.0, fs, 'normal')
    mel_spec = melspec_layer(audio.reshape(1, 1, -1)).numpy()
    if mel_spec.min()<-3:
        print(mel_spec.min(), mel_spec.max())

In [63]:
for audio_path in audio_paths:
    audio = load_audio(audio_path, 0.0, 0.0, 10.0, 0.0, fs, 'normal')
    x = melspec_layer.m(audio.reshape(1, 1, -1)) + 0.06
    x = tf.math.log(tf.maximum(x, melspec_layer.amin)) / math.log(10)
    x = x - tf.reduce_max(x)
    val = tf.reduce_min(x).numpy()
    if val < -5:
        print(val)

In [30]:
audio = load_audio(audio_paths[6], 0.0, 0.0, 10.0, 0.0, fs, 'normal')

# Amplitude Mel-Spectrogram
x = melspec_layer.m(audio.reshape(1, 1, -1))
# Clip x below from amin
x = tf.maximum(x, 1e-5)
# log-power Mel-spectrogram
x_ref = tf.reduce_max(x) # Reference is the maximum value of x
x = 10*tf.math.log(x**2/x_ref**2)/ tf.math.log(tf.constant(10, dtype=x.dtype))
# Clip x below from -dynamic_range dB
x = tf.maximum(x, -80)
# Normalize x to be in [0, 1]
x = 1 + (x/80)

In [32]:
print(tf.reduce_min(x), tf.reduce_max(x))

tf.Tensor(0.0, shape=(), dtype=float32) tf.Tensor(1.0, shape=(), dtype=float32)


In [39]:
cfg = load_config("default")
melspec_layer = get_melspec_layer(cfg)

print(cfg["MODEL"])

fs = cfg["MODEL"]["FS"]

cli: Configuration from ./config/default.yaml
{'FEAT': 'melspec', 'FS': 8000, 'DUR': 1.0, 'HOP': 0.5, 'STFT_WIN': 1024, 'STFT_HOP': 256, 'F_MIN': 300.0, 'F_MAX': 4000.0, 'N_MELS': 256, 'EMB_SZ': 128, 'BN': 'layer_norm2d', 'SEGMENT_NORM': True}


In [40]:
for audio_path in audio_paths:
    audio = load_audio(audio_path, 0.0, 0.0, 10.0, 0.0, fs, 'normal')
    mel_spec = melspec_layer(audio.reshape(1, 1, -1)).numpy()
    print(mel_spec.min(), mel_spec.max())


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

-1.0 1.0
-1.0 1.0
-1.0 1.0
-1.0 1.0
-1.0 1.0
-1.0 1.0
-1.0 1.0
-1.0 1.0
-1.0 1.0
-1.0 1.0
-1.0 1.0
-1.0 1.0
-1.0 1.0
-1.0 1.0
-1.0 1.0
-1.0 1.0
-1.0 1.0
-1.0 1.0
-1.0 1.0
-1.0 1.0
-1.0 1.0
-1.0 1.0
-1.0 1.0
-1.0 1.0
-1.0 1.0
-1.0 1.0
-1.0 1.0
-1.0 1.0
-1.0 1.0
-1.0 1.0
-1.0 1.0
-1.0 1.0
-1.0 1.0
-1.0 1.0
-1.0 1.0
-1.0 1.0
-1.0 1.0
-1.0 1.0
-1.0 1.0
-1.0 1.0
-1.0 1.0
-1.0 1.0
-1.0 1.0
-1.0 1.0
-1.0 1.0
-1.0 1.0
-1.0 1.0
-1.0 1.0
-1.0 1.0
-1.0 1.0
-1.0 1.0
-1.0 1.0
-1.0 1.0
-1.0 1.0
-1.0 1.0
-1.0 1.0
-1.0

KeyboardInterrupt: 

In [32]:
audio = load_audio(audio_paths[0], 0.0, 0.0, 10.0, 0.0, fs, 'normal')
mel_spec = melspec_layer(audio.reshape(1, 1, -1)).numpy()
mel_spec = mel_spec.reshape(mel_spec.shape[1], mel_spec.shape[2])
print(mel_spec.shape)
print(mel_spec.min(), mel_spec.max())

(256, 313)
-80.0 0.0


In [33]:
mel_spec_librosa = librosa.feature.melspectrogram(y=audio, 
                                                sr=fs, 
                                                n_fft=cfg["MODEL"]["STFT_WIN"], 
                                                hop_length=cfg["MODEL"]["STFT_HOP"],
                                                n_mels=cfg["MODEL"]["N_MELS"],
                                                fmin=cfg["MODEL"]["F_MIN"],
                                                fmax=cfg["MODEL"]["F_MAX"],
                                                center=True,
                                                #pad_mode="center",
                                                power=2)
print(mel_spec_librosa.shape)
mel_spec_librosa = 10*np.log10(mel_spec_librosa/np.max(mel_spec_librosa))
mel_spec_librosa = np.where(mel_spec_librosa>=-80, mel_spec_librosa, -80)
print(mel_spec_librosa.min(), mel_spec_librosa.max())

(256, 313)
-80.0 0.0


In [34]:
np.allclose(mel_spec, mel_spec_librosa, atol=1e-1)

False

In [35]:
mel_spec

array([[-36.68943 , -21.742113, -20.309765, ..., -19.822012, -20.489365,
        -24.145638],
       [-36.938984, -24.977938, -21.445936, ..., -27.986769, -28.033638,
        -29.870455],
       [-37.87073 , -30.565044, -24.64006 , ..., -26.313152, -25.039806,
        -24.922733],
       ...,
       [-79.64657 , -65.56791 , -56.17618 , ..., -55.69545 , -55.18178 ,
        -49.996162],
       [-80.      , -80.      , -75.46232 , ..., -66.33388 , -63.963314,
        -51.220177],
       [-80.      , -80.      , -80.      , ..., -80.      , -67.040184,
        -51.65086 ]], dtype=float32)

In [31]:
mel_spec_librosa

array([[-37.11386749, -22.11639493, -20.73415837, ..., -19.70487733,
        -20.10804385, -23.49306332],
       [-37.35740441, -25.02066466, -21.77221545, ..., -27.34627797,
        -28.37688437, -29.90850058],
       [-38.27259395, -30.67714086, -24.86745692, ..., -24.88787459,
        -24.69978753, -25.06100411],
       ...,
       [-77.952984  , -64.63177147, -55.13583889, ..., -55.4160699 ,
        -54.41510101, -50.21485826],
       [-80.        , -77.14191433, -71.42757918, ..., -64.99154154,
        -62.8093846 , -51.62960975],
       [-80.        , -80.        , -80.        , ..., -80.        ,
        -67.3180574 , -52.06569105]])